# SQL query from table names - Continued

In [15]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [16]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [17]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [18]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [19]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [20]:
#new
context_user = context.copy()
print(return_CCRMSQL("""YOUR QUERY HERE""", context_user))

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************rIQA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************rIQA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
#new
print(return_CCRMSQL("YOUR QUERY HERE", context_user))

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************rIQA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [21]:
#old
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************rIQA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [22]:
basic_context = [
    {'role': 'system', 'content': """
        You are an assistant that generates SQL queries. Keep the SQL simple.
        The database has three tables: employees, salary, and studies. Use these tables to answer user queries.
        """}
]

In [23]:
detailed_context = [
    {'role': 'system', 'content': """
        CREATE TABLE employees (
            ID_usr INT,
            name VARCHAR(100)
        );

        CREATE TABLE salary (
            ID_usr INT,
            year DATE,
            salary FLOAT
        );

        CREATE TABLE studies (
            ID INT,
            ID_usr INT,
            educational_level INT,
            Institution VARCHAR(100),
            Years DATE,
            Speciality VARCHAR(100)
        );
        """}
]

In [24]:
few_shot_context = [
    {'role': 'system', 'content': """
        CREATE TABLE employees (
            ID_usr INT,
            name VARCHAR(100)
        );

        CREATE TABLE salary (
            ID_usr INT,
            year DATE,
            salary FLOAT
        );

        CREATE TABLE studies (
            ID INT,
            ID_usr INT,
            educational_level INT,
            Institution VARCHAR(100),
            Years DATE,
            Speciality VARCHAR(100)
        );

        Example Query 1:
        Question: Who is the highest-paid employee?
        SQL: SELECT e.name FROM employees e JOIN salary s ON e.ID_usr = s.ID_usr ORDER BY s.salary DESC LIMIT 1;

        Example Query 2:
        Question: Which institution has the highest average salary?
        SQL: SELECT st.Institution, AVG(sa.salary) AS avg_salary FROM studies st JOIN employees e ON st.ID_usr = e.ID_usr JOIN salary sa ON e.ID_usr = sa.ID_usr GROUP BY st.Institution ORDER BY avg_salary DESC LIMIT 1;
        """}
]


Summarize findings based on the results:

Which variation performed best?
Did any variation generate hallucinated or incorrect SQL?
What did you learn about prompting GPT models for SQL?
Example Report Summary:
Findings:
Basic Prompt:

Strength: Simplicity.
Weakness: Occasionally produced incomplete or incorrect SQL due to lack of detailed schema.
Detailed Schema Prompt:

Strength: More consistent outputs due to explicit table structure.
Weakness: Still missed certain query nuances without examples.
Few-Shot Examples:

Strength: Provided accurate, contextually appropriate SQL queries.
Weakness: Slightly more verbose prompt, requiring additional setup time.
Key Learning:
Providing detailed schemas with examples (few-shot learning) significantly improves the quality and accuracy of generated SQL queries. This highlights the importance of crafting prompts tailored to specific tasks.